In [17]:
from scipy.io.arff import loadarff
import numpy as np
import pandas as pd

from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import f1_score

from imblearn.over_sampling import RandomOverSampler

In [2]:
path = '../../data/Change-Level-Prediction-Data-20191107T052353Z-001/Change-Level-Prediction-Data/lucene_data/0/'
fname = 'train.arff'

data, metadata = loadarff(path + fname)

In [3]:
type(data)

numpy.ndarray

In [4]:
type(metadata)

scipy.io.arff.arffread.MetaData

In [5]:
metadata

Dataset: lucene_crossvalidation
	change_id's type is numeric
	401_lines_added's type is numeric
	402_lines_deleted's type is numeric
	403_lines_changed's type is numeric
	404_loc's type is numeric
	405_file_lines's type is numeric
	406_time_of_day's type is numeric
	407_day_of_week's type is numeric
	408_previous_patches's type is numeric
	409_previous_buggy_patches's type is numeric
	410_file_age's type is numeric
	411_commit_time's type is nominal, range is ('2010-09-17 21:10:50+00:00', '0', '2010-09-17 10:30:03+00:00', '2010-09-17 21:33:48+00:00', '2010-09-18 11:50:31+00:00', '2010-09-19 15:40:06+00:00', '2010-09-20 17:27:15+00:00', '2010-09-21 01:10:37+00:00', '2010-09-21 13:40:35+00:00', '2010-09-22 15:14:20+00:00', '2010-09-23 12:29:40+00:00', '2010-09-23 19:59:55+00:00', '2010-09-24 01:08:23+00:00', '2010-09-24 06:09:45+00:00', '2010-09-24 18:29:02+00:00', '2010-09-25 19:32:37+00:00', '2010-09-27 10:32:27+00:00', '2010-09-27 16:10:29+00:00', '2010-09-29 04:11:29+00:00', '2010-09

In [163]:
np.unique(train_df['10069_version'])

array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
        11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  22.,
        23.,  24.,  25.,  32.,  33., 134., 135., 136., 137.])

So this probably isn't the software version number.

In [3]:
test_fname = 'test.arff'

In [4]:
test_data, test_metadata = loadarff(path + test_fname)

In [5]:
nb = GaussianNB()

In [6]:
train_df = pd.DataFrame(data)

In [7]:
test_df = pd.DataFrame(test_data)

In [30]:
train_df.head()

,change_id,401_lines_added,402_lines_deleted,403_lines_changed,404_loc,405_file_lines,406_time_of_day,407_day_of_week,408_previous_patches,409_previous_buggy_patches,...,115481_clonelock,115483_getreadcopi,115484_fromkei,19287_runbarewrapp,19306_checkedclass,19316_runbar,45787_restorepreflex,45788_installpreflexrw,45789_preflexsav,500_Buggy?
0,9007.0,0.0,0.0,8.0,10.0,0.0,21.0,5.0,8.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,b'0'
1,9016.0,0.0,5.0,10.0,20.0,0.0,21.0,5.0,10.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,b'0'
2,9002.0,1.0,0.0,4.0,6.0,1603.0,10.0,5.0,15.0,13.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,b'0'
3,9011.0,0.0,0.0,2.0,2.0,0.0,21.0,5.0,9.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,b'0'
4,9035.0,5.0,1.0,4.0,11.0,296.0,11.0,6.0,3.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,b'0'


In [12]:
train_df.columns[:10]

Index(['change_id', '401_lines_added', '402_lines_deleted',
       '403_lines_changed', '404_loc', '405_file_lines', '406_time_of_day',
       '407_day_of_week', '408_previous_patches',
       '409_previous_buggy_patches'],
      dtype='object')

In [8]:
train_df.drop(['411_commit_time'], axis=1, inplace=True)
test_df.drop(['411_commit_time'], axis=1, inplace=True)

train_df.dropna(inplace=True)
test_df.dropna(inplace=True)

In [9]:
x_train = train_df.drop('500_Buggy?', axis=1)
y_train = train_df['500_Buggy?']

In [10]:
x_test = test_df.drop('500_Buggy?', axis=1)
y_test = test_df['500_Buggy?']

In [15]:
(sum(y_train == b'1') / len(y_train)), (sum(y_test == b'1') / len(y_test))

(0.1009433962264151, 0.21494661921708186)

In [16]:
np.where(x_train.columns == '412_full_path')[0]

array([11])

In [58]:
# One-hot encode the filename column
x_train['train'] = 1
x_test['train'] = 0

combined = pd.concat([x_train, x_test])
df = pd.get_dummies(combined['412_full_path'])

combined = pd.concat([combined, df], axis=1)
x_train = combined[combined['train'] == 1]
x_test = combined[combined['train'] == 0]

x_train.drop('train', axis=1, inplace=True)
x_test.drop('train', axis=1, inplace=True)

/Users/ryedida/Library/Python/3.7/lib/python/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [18]:
x_train.drop('412_full_path', axis=1, inplace=True)
x_test.drop('412_full_path', axis=1, inplace=True)

In [19]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((1060, 21265), (1060,), (1405, 21265), (1405,))

In [19]:
np.where(np.array([x_train[x].dtype for x in x_train.columns]) not in ['float64', 'uint8'])

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [136]:
np.where(np.array([x_test[x].dtype for x in x_test.columns]) != 'float64')

(array([], dtype=int64),)

In [20]:
y_train[np.where(y_train == b'0')[0]] = 0
y_train[np.where(y_train == b'1')[0]] = 1

y_test[np.where(y_test == b'0')[0]] = 0
y_test[np.where(y_test == b'1')[0]] = 1

/Users/ryedida/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/ryedida/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/ryedida/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aft

In [22]:
sampler = RandomOverSampler()
x_train, y_train = sampler.fit_resample(x_train, y_train)

In [23]:
nb.fit(x_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [24]:
preds = nb.predict(x_test)
f1_score(y_test, preds)

0.2803738317757009

## Exploring the data

In [115]:
base_path = '../../data/Change-Level-Prediction-Data-20191107T052353Z-001/Change-Level-Prediction-Data/lucene_data/'

In [120]:
loadarff(base_path + '5/train.arff')[1]

Dataset: lucene_crossvalidation
	change_id's type is numeric
	401_lines_added's type is numeric
	402_lines_deleted's type is numeric
	403_lines_changed's type is numeric
	404_loc's type is numeric
	405_file_lines's type is numeric
	406_time_of_day's type is numeric
	407_day_of_week's type is numeric
	408_previous_patches's type is numeric
	409_previous_buggy_patches's type is numeric
	410_file_age's type is numeric
	411_commit_time's type is nominal, range is ('2011-02-14 14:14:10+00:00', '0', '2011-02-15 09:24:06+00:00', '2011-02-17 11:00:45+00:00', '2011-02-19 04:49:36+00:00', '2011-03-01 19:58:56+00:00', '2011-03-02 13:54:01+00:00', '2011-03-08 15:10:57+00:00', '2011-03-09 22:32:09+00:00', '2011-02-17 12:22:49+00:00', '2011-02-17 12:26:11+00:00', '2011-02-17 15:44:42+00:00', '2011-02-17 15:44:59+00:00', '2011-02-17 15:52:33+00:00', '2011-02-17 16:08:40+00:00', '2011-03-12 18:24:35+00:00', '2011-03-13 02:46:01+00:00', '2011-02-21 19:05:13+00:00', '2011-02-23 16:23:57+00:00', '2011-02

In [154]:
n_uniques = np.array([len(np.unique(train_df[x])) for x in train_df.columns])

In [156]:
len(np.where(n_uniques == 1)[0])

5861

In [160]:
x_train.drop(train_df.columns[np.where(n_uniques == 1)[0]], axis=1, inplace=True)
x_test.drop(test_df.columns[np.where(n_uniques == 1)[0]], axis=1, inplace=True)

In [161]:
x_train.shape

(1060, 15404)

In [162]:
nb = GaussianNB()
nb.fit(x_train, y_train)
nb.score(x_test, y_test)

0.45409252669039146